## Imports

In [1]:
# pip install --upgrade scikit-learn

In [2]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.4
Sklearn  0.24.2


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [3]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "your_kaggle_username"
    os.environ['KAGGLE_KEY']      = "your_kaggle_api_key"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "./titanic/"

## Load data

In [4]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [5]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [6]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [7]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)

get_Title_from_Name = lambda name: name.split(',')[1].split('.')[0].strip()

df['Title']      = df['Name'].map(get_Title_from_Name)
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)

In [8]:
df['Title'] 

PassengerId
1        Mr
2       Mrs
3      Miss
4       Mrs
5        Mr
       ... 
887     Rev
888    Miss
889    Miss
890      Mr
891      Mr
Name: Title, Length: 891, dtype: object

In [9]:
df_test['Title'] 

PassengerId
892         Mr
893        Mrs
894         Mr
895         Mr
896        Mrs
         ...  
1305        Mr
1306      Dona
1307        Mr
1308        Mr
1309    Master
Name: Title, Length: 418, dtype: object

In [10]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [11]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [12]:
df['Title'].values[886]


'Rev'

In [13]:
df["Title"] 

PassengerId
1        Mr
2       Mrs
3      Miss
4       Mrs
5        Mr
       ... 
887     Rev
888    Miss
889    Miss
890      Mr
891      Mr
Name: Title, Length: 891, dtype: object

In [14]:
# Use map to apply the prevous dict

df["Title"] = df.Title.map(title_dictionary) 
df_test["Title"] = df_test.Title.map(title_dictionary)

# YOUR CODE HERE
# raise NotImplementedError()

In [15]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [16]:
df_test['Ticket']

PassengerId
892                 330911
893                 363272
894                 240276
895                 315154
896                3101298
               ...        
1305             A.5. 3236
1306              PC 17758
1307    SOTON/O.Q. 3101262
1308                359309
1309                  2668
Name: Ticket, Length: 418, dtype: object

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [17]:
df_test['Cabin']

PassengerId
892      NaN
893      NaN
894      NaN
895      NaN
896      NaN
        ... 
1305     NaN
1306    C105
1307     NaN
1308     NaN
1309     NaN
Name: Cabin, Length: 418, dtype: object

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [18]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [19]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [20]:
num_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False))
])

cat_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
  ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

# YOUR CODE HERE
# raise NotImplementedError()

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked', 'Title'])])

In [21]:
assert type(tree_prepro)      is compose._column_transformer.ColumnTransformer
assert type(num_4_treeModels) is pipeline.Pipeline
assert type(cat_4_treeModels) is pipeline.Pipeline
assert len(num_4_treeModels) == 1
assert len(cat_4_treeModels) == 2

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [22]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [40]:

tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(n_estimators=200, random_state=10,learning_rate=0.1),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":GradientBoostingClassifier(),
  "XGBoost":XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()
}

# YOUR CODE HERE
# raise NotImplementedError()

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [24]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [25]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y,
    test_size=0.2,
    stratify = y,   
    random_state=4 )

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = model.predict(x_val)
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

/home/nsarkhanov/anaconda3/envs/aienv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:34:26] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.008911
0:	learn: 0.6874585	total: 51.9ms	remaining: 51.8s
1:	learn: 0.6817921	total: 55.1ms	remaining: 27.5s
2:	learn: 0.6768098	total: 56.5ms	remaining: 18.8s
3:	learn: 0.6720448	total: 58.3ms	remaining: 14.5s
4:	learn: 0.6669187	total: 60ms	remaining: 11.9s
5:	learn: 0.6622295	total: 63.1ms	remaining: 10.4s
6:	learn: 0.6573380	total: 66ms	remaining: 9.36s
7:	learn: 0.6524357	total: 67.8ms	remaining: 8.41s
8:	learn: 0.6468926	total: 69.7ms	remaining: 7.68s
9:	learn: 0.6418077	total: 71ms	remaining: 7.03s
10:	learn: 0.6366277	total: 73.8ms	remaining: 6.63s
11:	learn: 0.6317214	total: 76.1ms	remaining: 6.26s
12:	learn: 0.6279207	total: 77.9ms	remaining: 5.91s
13:	learn: 0.6242446	total: 79.8ms	remaining: 5.62s
14:	le

215:	learn: 0.3828877	total: 377ms	remaining: 1.37s
216:	learn: 0.3825339	total: 380ms	remaining: 1.37s
217:	learn: 0.3823418	total: 384ms	remaining: 1.38s
218:	learn: 0.3820418	total: 391ms	remaining: 1.4s
219:	learn: 0.3815158	total: 393ms	remaining: 1.39s
220:	learn: 0.3812077	total: 410ms	remaining: 1.45s
221:	learn: 0.3809243	total: 413ms	remaining: 1.45s
222:	learn: 0.3806912	total: 414ms	remaining: 1.44s
223:	learn: 0.3805121	total: 416ms	remaining: 1.44s
224:	learn: 0.3801366	total: 418ms	remaining: 1.44s
225:	learn: 0.3797622	total: 425ms	remaining: 1.46s
226:	learn: 0.3794836	total: 428ms	remaining: 1.46s
227:	learn: 0.3790422	total: 435ms	remaining: 1.47s
228:	learn: 0.3788199	total: 440ms	remaining: 1.48s
229:	learn: 0.3787534	total: 444ms	remaining: 1.49s
230:	learn: 0.3786067	total: 450ms	remaining: 1.5s
231:	learn: 0.3782604	total: 452ms	remaining: 1.5s
232:	learn: 0.3779846	total: 454ms	remaining: 1.49s
233:	learn: 0.3777281	total: 467ms	remaining: 1.53s
234:	learn: 0.3

391:	learn: 0.3500002	total: 758ms	remaining: 1.18s
392:	learn: 0.3498841	total: 760ms	remaining: 1.17s
393:	learn: 0.3498342	total: 761ms	remaining: 1.17s
394:	learn: 0.3496911	total: 762ms	remaining: 1.17s
395:	learn: 0.3494119	total: 763ms	remaining: 1.16s
396:	learn: 0.3491791	total: 764ms	remaining: 1.16s
397:	learn: 0.3490898	total: 766ms	remaining: 1.16s
398:	learn: 0.3490476	total: 767ms	remaining: 1.16s
399:	learn: 0.3490093	total: 768ms	remaining: 1.15s
400:	learn: 0.3488302	total: 769ms	remaining: 1.15s
401:	learn: 0.3487285	total: 773ms	remaining: 1.15s
402:	learn: 0.3485355	total: 774ms	remaining: 1.15s
403:	learn: 0.3483640	total: 776ms	remaining: 1.14s
404:	learn: 0.3482161	total: 777ms	remaining: 1.14s
405:	learn: 0.3481629	total: 778ms	remaining: 1.14s
406:	learn: 0.3479763	total: 780ms	remaining: 1.14s
407:	learn: 0.3477383	total: 781ms	remaining: 1.13s
408:	learn: 0.3475196	total: 782ms	remaining: 1.13s
409:	learn: 0.3473947	total: 782ms	remaining: 1.13s
410:	learn: 

631:	learn: 0.3209187	total: 1.13s	remaining: 655ms
632:	learn: 0.3207711	total: 1.13s	remaining: 653ms
633:	learn: 0.3205822	total: 1.13s	remaining: 652ms
634:	learn: 0.3205237	total: 1.13s	remaining: 651ms
635:	learn: 0.3204768	total: 1.14s	remaining: 650ms
636:	learn: 0.3203831	total: 1.14s	remaining: 649ms
637:	learn: 0.3203519	total: 1.14s	remaining: 647ms
638:	learn: 0.3202068	total: 1.14s	remaining: 645ms
639:	learn: 0.3201873	total: 1.14s	remaining: 643ms
640:	learn: 0.3200130	total: 1.14s	remaining: 641ms
641:	learn: 0.3199202	total: 1.15s	remaining: 639ms
642:	learn: 0.3198245	total: 1.15s	remaining: 637ms
643:	learn: 0.3197703	total: 1.15s	remaining: 635ms
644:	learn: 0.3196071	total: 1.15s	remaining: 633ms
645:	learn: 0.3195051	total: 1.15s	remaining: 632ms
646:	learn: 0.3193927	total: 1.16s	remaining: 635ms
647:	learn: 0.3191539	total: 1.17s	remaining: 633ms
648:	learn: 0.3190506	total: 1.17s	remaining: 631ms
649:	learn: 0.3188992	total: 1.17s	remaining: 629ms
650:	learn: 

820:	learn: 0.2998720	total: 1.5s	remaining: 326ms
821:	learn: 0.2997991	total: 1.5s	remaining: 325ms
822:	learn: 0.2996552	total: 1.5s	remaining: 323ms
823:	learn: 0.2994864	total: 1.5s	remaining: 321ms
824:	learn: 0.2994202	total: 1.5s	remaining: 319ms
825:	learn: 0.2993233	total: 1.51s	remaining: 317ms
826:	learn: 0.2992880	total: 1.51s	remaining: 316ms
827:	learn: 0.2992654	total: 1.51s	remaining: 314ms
828:	learn: 0.2991172	total: 1.51s	remaining: 312ms
829:	learn: 0.2990408	total: 1.51s	remaining: 310ms
830:	learn: 0.2989697	total: 1.51s	remaining: 308ms
831:	learn: 0.2988655	total: 1.51s	remaining: 306ms
832:	learn: 0.2988210	total: 1.52s	remaining: 304ms
833:	learn: 0.2987706	total: 1.52s	remaining: 302ms
834:	learn: 0.2985850	total: 1.52s	remaining: 301ms
835:	learn: 0.2983609	total: 1.52s	remaining: 299ms
836:	learn: 0.2982989	total: 1.52s	remaining: 297ms
837:	learn: 0.2982058	total: 1.52s	remaining: 295ms
838:	learn: 0.2979871	total: 1.53s	remaining: 293ms
839:	learn: 0.297

,Model,Accuracy,Bal Acc.,Time
1,CatBoost,86.592179,84.769433,2.012235
2,Skl GBM,86.033520,84.044796,0.146765
3,Skl HistGBM,86.033520,84.044796,0.155562
4,XGBoost,84.916201,84.216074,0.931210
5,AdaBoost,83.798883,83.036891,0.125515
6,LightGBM,83.240223,82.042161,1.181260
7,Random Forest,82.681564,81.047431,0.223016
8,Extra Trees,79.329609,77.779974,0.186413
9,Decision Tree,76.536313,75.777339,0.020404


In [26]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [42]:
from sklearn.model_selection import cross_val_predict

skf = model_selection.StratifiedKFold(
    n_splits=10, shuffle=True, random_state=0
)

# YOUR CODE HERE
# raise NotImplementedError()


results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})


for model_name, model in tree_classifiers.items():

        
    start_time = time.time()
    model.fit(x_train, y_train)
    
    total_time = time.time() - start_time
    pred = cross_val_predict(model, x_val, y_val, cv=5)

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              


# YOUR CODE HERE
#raise NotImplementedError()


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

/home/nsarkhanov/anaconda3/envs/aienv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:46:53] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nsarkhanov/anaconda3/envs/aienv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nsarkhanov/anaconda3/envs/aienv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:46:54] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:46:54] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nsarkhanov/anaconda3/envs/aienv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nsarkhanov/anaconda3/envs/aienv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

303:	learn: 0.3630179	total: 241ms	remaining: 551ms
304:	learn: 0.3626342	total: 241ms	remaining: 549ms
305:	learn: 0.3624215	total: 242ms	remaining: 548ms
306:	learn: 0.3620151	total: 242ms	remaining: 547ms
307:	learn: 0.3615600	total: 243ms	remaining: 547ms
308:	learn: 0.3612996	total: 244ms	remaining: 546ms
309:	learn: 0.3608147	total: 245ms	remaining: 545ms
310:	learn: 0.3604304	total: 246ms	remaining: 544ms
311:	learn: 0.3600032	total: 246ms	remaining: 543ms
312:	learn: 0.3595322	total: 247ms	remaining: 543ms
313:	learn: 0.3591045	total: 248ms	remaining: 542ms
314:	learn: 0.3587535	total: 249ms	remaining: 541ms
315:	learn: 0.3584022	total: 249ms	remaining: 539ms
316:	learn: 0.3581602	total: 250ms	remaining: 538ms
317:	learn: 0.3577026	total: 250ms	remaining: 537ms
318:	learn: 0.3571767	total: 251ms	remaining: 535ms
319:	learn: 0.3569383	total: 251ms	remaining: 534ms
320:	learn: 0.3565924	total: 252ms	remaining: 534ms
321:	learn: 0.3563006	total: 253ms	remaining: 533ms
322:	learn: 

540:	learn: 0.2949010	total: 423ms	remaining: 359ms
541:	learn: 0.2947668	total: 424ms	remaining: 358ms
542:	learn: 0.2945566	total: 425ms	remaining: 357ms
543:	learn: 0.2943271	total: 425ms	remaining: 357ms
544:	learn: 0.2942062	total: 427ms	remaining: 356ms
545:	learn: 0.2940012	total: 427ms	remaining: 355ms
546:	learn: 0.2936394	total: 429ms	remaining: 355ms
547:	learn: 0.2935217	total: 429ms	remaining: 354ms
548:	learn: 0.2933682	total: 430ms	remaining: 353ms
549:	learn: 0.2929497	total: 431ms	remaining: 353ms
550:	learn: 0.2927639	total: 432ms	remaining: 352ms
551:	learn: 0.2925985	total: 440ms	remaining: 357ms
552:	learn: 0.2923434	total: 441ms	remaining: 357ms
553:	learn: 0.2921103	total: 442ms	remaining: 356ms
554:	learn: 0.2918669	total: 443ms	remaining: 355ms
555:	learn: 0.2914537	total: 444ms	remaining: 355ms
556:	learn: 0.2913629	total: 445ms	remaining: 354ms
557:	learn: 0.2912152	total: 446ms	remaining: 353ms
558:	learn: 0.2909687	total: 447ms	remaining: 353ms
559:	learn: 

753:	learn: 0.2584987	total: 606ms	remaining: 198ms
754:	learn: 0.2583530	total: 607ms	remaining: 197ms
755:	learn: 0.2582345	total: 608ms	remaining: 196ms
756:	learn: 0.2581216	total: 609ms	remaining: 195ms
757:	learn: 0.2580032	total: 610ms	remaining: 195ms
758:	learn: 0.2579480	total: 610ms	remaining: 194ms
759:	learn: 0.2577574	total: 611ms	remaining: 193ms
760:	learn: 0.2576327	total: 612ms	remaining: 192ms
761:	learn: 0.2575022	total: 612ms	remaining: 191ms
762:	learn: 0.2574021	total: 613ms	remaining: 191ms
763:	learn: 0.2572265	total: 614ms	remaining: 190ms
764:	learn: 0.2571952	total: 617ms	remaining: 189ms
765:	learn: 0.2571200	total: 618ms	remaining: 189ms
766:	learn: 0.2571096	total: 619ms	remaining: 188ms
767:	learn: 0.2570388	total: 619ms	remaining: 187ms
768:	learn: 0.2569330	total: 620ms	remaining: 186ms
769:	learn: 0.2567362	total: 621ms	remaining: 186ms
770:	learn: 0.2565282	total: 624ms	remaining: 185ms
771:	learn: 0.2564787	total: 626ms	remaining: 185ms
772:	learn: 

971:	learn: 0.2305694	total: 785ms	remaining: 22.6ms
972:	learn: 0.2303315	total: 787ms	remaining: 21.8ms
973:	learn: 0.2302176	total: 787ms	remaining: 21ms
974:	learn: 0.2301783	total: 788ms	remaining: 20.2ms
975:	learn: 0.2300621	total: 790ms	remaining: 19.4ms
976:	learn: 0.2300354	total: 791ms	remaining: 18.6ms
977:	learn: 0.2299465	total: 792ms	remaining: 17.8ms
978:	learn: 0.2298632	total: 793ms	remaining: 17ms
979:	learn: 0.2298189	total: 793ms	remaining: 16.2ms
980:	learn: 0.2296387	total: 794ms	remaining: 15.4ms
981:	learn: 0.2295941	total: 794ms	remaining: 14.6ms
982:	learn: 0.2295267	total: 795ms	remaining: 13.7ms
983:	learn: 0.2293626	total: 796ms	remaining: 12.9ms
984:	learn: 0.2292447	total: 797ms	remaining: 12.1ms
985:	learn: 0.2291602	total: 798ms	remaining: 11.3ms
986:	learn: 0.2290910	total: 800ms	remaining: 10.5ms
987:	learn: 0.2288429	total: 801ms	remaining: 9.72ms
988:	learn: 0.2286113	total: 801ms	remaining: 8.91ms
989:	learn: 0.2285226	total: 802ms	remaining: 8.1m

,Model,Accuracy,Bal Acc.,Time
1,Decision Tree,84.357542,82.681159,0.020015
2,CatBoost,81.564246,79.598155,1.939679
3,XGBoost,81.005587,79.143610,0.568037
4,LightGBM,81.005587,78.873518,0.165740
5,Skl HistGBM,80.446927,79.229249,0.132381
6,Skl GBM,79.329609,78.320158,0.131659
7,Extra Trees,78.770950,77.055336,0.190957
8,Random Forest,78.770950,77.325428,0.241923
9,AdaBoost,75.977654,74.512516,0.486172


In [41]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

AssertionError: 

# Exercise 7.1
Train with all data the best model

In [53]:
best_model = tree_classifiers["CatBoost"]

best_model.fit(x,y)

# YOUR CODE HERE
# raise NotImplementedError()

Learning rate set to 0.009807
0:	learn: 0.6861863	total: 4.96ms	remaining: 4.96s
1:	learn: 0.6799925	total: 7.09ms	remaining: 3.54s
2:	learn: 0.6738277	total: 13.5ms	remaining: 4.47s
3:	learn: 0.6673321	total: 18.1ms	remaining: 4.51s
4:	learn: 0.6616071	total: 21.3ms	remaining: 4.23s
5:	learn: 0.6565227	total: 22.7ms	remaining: 3.77s
6:	learn: 0.6511753	total: 24.2ms	remaining: 3.44s
7:	learn: 0.6458359	total: 25.7ms	remaining: 3.19s
8:	learn: 0.6400293	total: 27.3ms	remaining: 3.01s
9:	learn: 0.6345222	total: 29.5ms	remaining: 2.92s
10:	learn: 0.6289567	total: 31.1ms	remaining: 2.8s
11:	learn: 0.6235886	total: 33.7ms	remaining: 2.78s
12:	learn: 0.6188597	total: 35.2ms	remaining: 2.67s
13:	learn: 0.6150559	total: 35.9ms	remaining: 2.53s
14:	learn: 0.6106695	total: 37.1ms	remaining: 2.44s
15:	learn: 0.6068261	total: 37.7ms	remaining: 2.32s
16:	learn: 0.6017974	total: 39.3ms	remaining: 2.27s
17:	learn: 0.5969672	total: 42.4ms	remaining: 2.31s
18:	learn: 0.5928576	total: 43.6ms	remaining:

239:	learn: 0.3699601	total: 373ms	remaining: 1.18s
240:	learn: 0.3697160	total: 374ms	remaining: 1.18s
241:	learn: 0.3694978	total: 376ms	remaining: 1.18s
242:	learn: 0.3693361	total: 377ms	remaining: 1.18s
243:	learn: 0.3690897	total: 379ms	remaining: 1.17s
244:	learn: 0.3690576	total: 380ms	remaining: 1.17s
245:	learn: 0.3689091	total: 381ms	remaining: 1.17s
246:	learn: 0.3685930	total: 382ms	remaining: 1.16s
247:	learn: 0.3683769	total: 383ms	remaining: 1.16s
248:	learn: 0.3681233	total: 385ms	remaining: 1.16s
249:	learn: 0.3679391	total: 387ms	remaining: 1.16s
250:	learn: 0.3676866	total: 389ms	remaining: 1.16s
251:	learn: 0.3674644	total: 390ms	remaining: 1.16s
252:	learn: 0.3673463	total: 392ms	remaining: 1.16s
253:	learn: 0.3671709	total: 395ms	remaining: 1.16s
254:	learn: 0.3670035	total: 397ms	remaining: 1.16s
255:	learn: 0.3667260	total: 399ms	remaining: 1.16s
256:	learn: 0.3666246	total: 400ms	remaining: 1.16s
257:	learn: 0.3663372	total: 402ms	remaining: 1.16s
258:	learn: 

511:	learn: 0.3324237	total: 753ms	remaining: 717ms
512:	learn: 0.3322839	total: 754ms	remaining: 716ms
513:	learn: 0.3321931	total: 755ms	remaining: 714ms
514:	learn: 0.3321575	total: 756ms	remaining: 712ms
515:	learn: 0.3321196	total: 758ms	remaining: 711ms
516:	learn: 0.3319740	total: 759ms	remaining: 709ms
517:	learn: 0.3318152	total: 760ms	remaining: 707ms
518:	learn: 0.3316605	total: 761ms	remaining: 706ms
519:	learn: 0.3315825	total: 763ms	remaining: 704ms
520:	learn: 0.3315217	total: 763ms	remaining: 702ms
521:	learn: 0.3313317	total: 764ms	remaining: 700ms
522:	learn: 0.3312519	total: 765ms	remaining: 698ms
523:	learn: 0.3311741	total: 767ms	remaining: 697ms
524:	learn: 0.3309857	total: 768ms	remaining: 695ms
525:	learn: 0.3308585	total: 769ms	remaining: 693ms
526:	learn: 0.3306655	total: 770ms	remaining: 691ms
527:	learn: 0.3304457	total: 772ms	remaining: 690ms
528:	learn: 0.3301529	total: 773ms	remaining: 688ms
529:	learn: 0.3301107	total: 774ms	remaining: 687ms
530:	learn: 

752:	learn: 0.3059293	total: 1.13s	remaining: 372ms
753:	learn: 0.3058785	total: 1.13s	remaining: 370ms
754:	learn: 0.3057487	total: 1.14s	remaining: 369ms
755:	learn: 0.3055408	total: 1.14s	remaining: 367ms
756:	learn: 0.3053406	total: 1.14s	remaining: 366ms
757:	learn: 0.3051834	total: 1.14s	remaining: 364ms
758:	learn: 0.3050827	total: 1.14s	remaining: 363ms
759:	learn: 0.3050217	total: 1.14s	remaining: 361ms
760:	learn: 0.3049061	total: 1.14s	remaining: 360ms
761:	learn: 0.3048234	total: 1.15s	remaining: 358ms
762:	learn: 0.3048108	total: 1.15s	remaining: 357ms
763:	learn: 0.3047226	total: 1.15s	remaining: 355ms
764:	learn: 0.3046798	total: 1.15s	remaining: 354ms
765:	learn: 0.3045224	total: 1.15s	remaining: 352ms
766:	learn: 0.3044452	total: 1.15s	remaining: 351ms
767:	learn: 0.3043936	total: 1.16s	remaining: 349ms
768:	learn: 0.3042321	total: 1.16s	remaining: 348ms
769:	learn: 0.3041494	total: 1.16s	remaining: 346ms
770:	learn: 0.3039984	total: 1.16s	remaining: 344ms
771:	learn: 

967:	learn: 0.2823625	total: 1.51s	remaining: 49.9ms
968:	learn: 0.2822576	total: 1.51s	remaining: 48.3ms
969:	learn: 0.2820042	total: 1.51s	remaining: 46.8ms
970:	learn: 0.2818532	total: 1.51s	remaining: 45.2ms
971:	learn: 0.2817720	total: 1.51s	remaining: 43.6ms
972:	learn: 0.2815659	total: 1.52s	remaining: 42.1ms
973:	learn: 0.2815061	total: 1.52s	remaining: 40.5ms
974:	learn: 0.2813209	total: 1.52s	remaining: 39ms
975:	learn: 0.2812482	total: 1.52s	remaining: 37.5ms
976:	learn: 0.2810643	total: 1.53s	remaining: 35.9ms
977:	learn: 0.2810017	total: 1.53s	remaining: 34.4ms
978:	learn: 0.2809162	total: 1.53s	remaining: 32.8ms
979:	learn: 0.2807662	total: 1.53s	remaining: 31.3ms
980:	learn: 0.2806655	total: 1.53s	remaining: 29.7ms
981:	learn: 0.2806063	total: 1.53s	remaining: 28.2ms
982:	learn: 0.2805396	total: 1.54s	remaining: 26.6ms
983:	learn: 0.2804744	total: 1.54s	remaining: 25ms
984:	learn: 0.2804075	total: 1.54s	remaining: 23.5ms
985:	learn: 0.2803553	total: 1.54s	remaining: 21.9

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('catboostclassifier',
                 <catboost.core.CatBoostClassifier object at 0x7fd5db769670>)])

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [54]:
test_pred = best_model.predict(x_test)

# YOUR CODE HERE
# raise NotImplementedError()

In [55]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [56]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [57]:
sub.to_csv("sub.csv")

In [52]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"

Traceback (most recent call last):
  File "/home/nsarkhanov/anaconda3/envs/aienv/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/home/nsarkhanov/anaconda3/envs/aienv/lib/python3.8/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/home/nsarkhanov/anaconda3/envs/aienv/lib/python3.8/site-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /home/nsarkhanov/.kaggle. Or use the environment method.
